In [160]:
import requests
from requests.exceptions import HTTPError, RequestException, Timeout
import json
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


In [161]:
URL1 = 'https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json'
data = None

try:
    resposta = requests.get(URL1, timeout=10)
    resposta.raise_for_status()

except HTTPError as http_err:
    print(f'Erro HTTP: {http_err}')

except ConnectionError as conn_err:
    print(f'Erro de conexão: {conn_err}')

except Timeout as timeout_err:
    print(f'Tempo de resposta excedido: {timeout_err}')

except RequestException as req_err:
    print(f'Erro na requisição: {req_err}')

except Exception as exc:
    print(f'Ocorreu um erro inesperado: {exc}')

else:
    print('Requisição bem-sucedida!')
    data = json.loads(resposta.text)

Requisição bem-sucedida!


In [162]:
deliveries_df = pd.DataFrame(data)
deliveries_df.head()

,name,region,origin,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


In [163]:
origin_df = pd.json_normalize(deliveries_df['origin'])
origin_df.head()

,lng,lat
0,-48.054989,-15.838145
1,-48.054989,-15.838145
2,-48.054989,-15.838145
3,-47.893662,-15.805118
4,-48.054989,-15.838145


In [164]:
deliveries_df['origin'] = pd.DataFrame(deliveries_df['origin'].apply(lambda value: str(value['lat']) + ', ' + str(value['lng'])))
deliveries_df.head()

,name,region,origin,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,"-15.83814451122274, -48.05498915846707",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,"-15.83814451122274, -48.05498915846707",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,"-15.83814451122274, -48.05498915846707",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,"-15.80511751066334, -47.89366206897872",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,"-15.83814451122274, -48.05498915846707",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


In [165]:
deliveries_df = pd.merge(left=deliveries_df, right=origin_df, how='inner', left_index=True, right_index=True)
deliveries_df.head()

,name,region,origin,vehicle_capacity,deliveries,lng,lat
0,cvrp-2-df-33,df-2,"-15.83814451122274, -48.05498915846707",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p...",-48.054989,-15.838145
1,cvrp-2-df-73,df-2,"-15.83814451122274, -48.05498915846707",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po...",-48.054989,-15.838145
2,cvrp-2-df-20,df-2,"-15.83814451122274, -48.05498915846707",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p...",-48.054989,-15.838145
3,cvrp-1-df-71,df-1,"-15.80511751066334, -47.89366206897872",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p...",-47.893662,-15.805118
4,cvrp-2-df-87,df-2,"-15.83814451122274, -48.05498915846707",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p...",-48.054989,-15.838145


In [166]:
deliveries_df.rename(columns={'lng': 'origin_lng', 'lat': 'origin_lat'}, inplace=True)
deliveries_df.head()

,name,region,origin,vehicle_capacity,deliveries,origin_lng,origin_lat
0,cvrp-2-df-33,df-2,"-15.83814451122274, -48.05498915846707",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p...",-48.054989,-15.838145
1,cvrp-2-df-73,df-2,"-15.83814451122274, -48.05498915846707",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po...",-48.054989,-15.838145
2,cvrp-2-df-20,df-2,"-15.83814451122274, -48.05498915846707",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p...",-48.054989,-15.838145
3,cvrp-1-df-71,df-1,"-15.80511751066334, -47.89366206897872",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p...",-47.893662,-15.805118
4,cvrp-2-df-87,df-2,"-15.83814451122274, -48.05498915846707",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p...",-48.054989,-15.838145


In [167]:
deliveries_exploded_df = deliveries_df[['deliveries']].explode('deliveries')
deliveries_exploded_df.head()

,deliveries
0,"{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'po..."
0,"{'id': '320c94b17aa685c939b3f3244c3099de', 'po..."
0,"{'id': '3663b42f4b8decb33059febaba46d5c8', 'po..."
0,"{'id': 'e11ab58363c38d6abc90d5fba87b7d7', 'poi..."
0,"{'id': '54cb45b7bbbd4e34e7150900f92d7f4b', 'po..."


In [168]:
deliveries_normalized_df = pd.concat([
    pd.DataFrame(deliveries_exploded_df['deliveries'].apply(lambda value: value['size'])).rename(columns={'deliveries': 'delivery_size'}),
    pd.DataFrame(deliveries_exploded_df['deliveries'].apply(lambda value: value['point']['lng'])).rename(columns={'deliveries': 'destiny_lng'}),
    pd.DataFrame(deliveries_exploded_df['deliveries'].apply(lambda value: value['point']['lat'])).rename(columns={'deliveries': 'destiny_lat'}),
], axis=1)

deliveries_normalized_df.head()

,delivery_size,destiny_lng,destiny_lat
0,9,-48.116189,-15.848929
0,2,-48.118195,-15.850772
0,1,-48.112483,-15.847871
0,2,-48.118023,-15.846471
0,7,-48.114898,-15.858055


In [169]:
deliveries_df = deliveries_df.drop('deliveries', axis=1)
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_normalized_df, how='right', left_index=True, right_index=True)
deliveries_df.head()

,name,region,origin,vehicle_capacity,origin_lng,origin_lat,delivery_size,destiny_lng,destiny_lat
0,cvrp-2-df-33,df-2,"-15.83814451122274, -48.05498915846707",180,-48.054989,-15.838145,9,-48.116189,-15.848929
0,cvrp-2-df-33,df-2,"-15.83814451122274, -48.05498915846707",180,-48.054989,-15.838145,2,-48.118195,-15.850772
0,cvrp-2-df-33,df-2,"-15.83814451122274, -48.05498915846707",180,-48.054989,-15.838145,1,-48.112483,-15.847871
0,cvrp-2-df-33,df-2,"-15.83814451122274, -48.05498915846707",180,-48.054989,-15.838145,2,-48.118023,-15.846471
0,cvrp-2-df-33,df-2,"-15.83814451122274, -48.05498915846707",180,-48.054989,-15.838145,7,-48.114898,-15.858055


In [170]:
deliveries_df = deliveries_df[['name', 'region', 'origin_lat', 'origin_lng', 'origin', 'vehicle_capacity', 'delivery_size', 'destiny_lat', 'destiny_lng']]
deliveries_df.reset_index(inplace=True, drop=True)
deliveries_df.head()

,name,region,origin_lat,origin_lng,origin,vehicle_capacity,delivery_size,destiny_lat,destiny_lng
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,"-15.83814451122274, -48.05498915846707",180,9,-15.848929,-48.116189
1,cvrp-2-df-33,df-2,-15.838145,-48.054989,"-15.83814451122274, -48.05498915846707",180,2,-15.850772,-48.118195
2,cvrp-2-df-33,df-2,-15.838145,-48.054989,"-15.83814451122274, -48.05498915846707",180,1,-15.847871,-48.112483
3,cvrp-2-df-33,df-2,-15.838145,-48.054989,"-15.83814451122274, -48.05498915846707",180,2,-15.846471,-48.118023
4,cvrp-2-df-33,df-2,-15.838145,-48.054989,"-15.83814451122274, -48.05498915846707",180,7,-15.858055,-48.114898


In [171]:
hub_df = deliveries_df[['region', 'origin']]
hub_df = hub_df.drop_duplicates().sort_values(by='region').reset_index(drop=True)
hub_df.head()

,region,origin
0,df-0,"-15.657013854445248, -47.802664728268745"
1,df-1,"-15.80511751066334, -47.89366206897872"
2,df-2,"-15.83814451122274, -48.05498915846707"


In [172]:
geolocator = Nominatim(user_agent='geoloc/1.0 (tiicoliro@gmail.com)')
geocoder = RateLimiter(geolocator.reverse, min_delay_seconds=1)

hub_df['geodata'] = hub_df['origin'].apply(geocoder)
hub_df.head()

,region,origin,geodata
0,df-0,"-15.657013854445248, -47.802664728268745","(Rua 7, Quadra 2, Sobradinho, Região Geográfic..."
1,df-1,"-15.80511751066334, -47.89366206897872","(SQS 303, Asa Sul, Brasília, Plano Piloto, Reg..."
2,df-2,"-15.83814451122274, -48.05498915846707","(Armazém do Bolo, lote 4/8, CSB 4/5, Taguating..."


In [173]:
hub_geodata_df = pd.json_normalize(hub_df['geodata'].apply(lambda data: data.raw))
hub_geodata_df = hub_geodata_df[["address.town", "address.suburb", "address.city"]]
hub_geodata_df.rename(columns={"address.town": "origin_town", "address.suburb": "origin_suburb", "address.city": "origin_city"}, inplace=True)
hub_geodata_df["origin_city"] = np.where(hub_geodata_df["origin_city"].notna(), hub_geodata_df["origin_city"], hub_geodata_df["origin_town"])
hub_geodata_df["origin_suburb"] = np.where(hub_geodata_df["origin_suburb"].notna(), hub_geodata_df["origin_suburb"], hub_geodata_df["origin_city"])
hub_geodata_df = hub_geodata_df.drop("origin_town", axis=1)
hub_geodata_df.head()

,origin_suburb,origin_city
0,Sobradinho,Sobradinho
1,Asa Sul,Brasília
2,Taguatinga,Taguatinga


In [174]:
hub_df = pd.merge(left=hub_df, right=hub_geodata_df, left_index=True, right_index=True)
hub_df = hub_df[["region", "origin_suburb", "origin_city"]]
hub_df.head()

,region,origin_suburb,origin_city
0,df-0,Sobradinho,Sobradinho
1,df-1,Asa Sul,Brasília
2,df-2,Taguatinga,Taguatinga


In [175]:
deliveries_df = pd.merge(left=deliveries_df, right=hub_df, how="inner", on="region")

In [176]:
deliveries_df = deliveries_df[['name', 'region', 'origin_lat', 'origin_lng', 'origin_suburb', 'origin_city', 'vehicle_capacity', 'delivery_size', 'destiny_lat', 'destiny_lng']]
deliveries_df.head()

,name,region,origin_lat,origin_lng,origin_suburb,origin_city,vehicle_capacity,delivery_size,destiny_lat,destiny_lng
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,9,-15.848929,-48.116189
1,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,2,-15.850772,-48.118195
2,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,1,-15.847871,-48.112483
3,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,2,-15.846471,-48.118023
4,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,7,-15.858055,-48.114898


In [177]:
URL2 = 'https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries-geodata.csv'

try:
    resposta = requests.get(URL2, timeout=10)
    resposta.raise_for_status()

except HTTPError as http_err:
    print(f'Erro HTTP: {http_err}')

except ConnectionError as conn_err:
    print(f'Erro de conexão: {conn_err}')

except Timeout as timeout_err:
    print(f'Tempo de resposta excedido: {timeout_err}')

except RequestException as req_err:
    print(f'Erro na requisição: {req_err}')

except Exception as exc:
    print(f'Ocorreu um erro inesperado: {exc}')

else:
    print('Requisição bem-sucedida!')
    
    output_file = 'destiny_geodata.csv'
    with open(output_file, mode='w', encoding='utf-8') as file:
        file.write(resposta.text)
    
    print(f'Dados salvos com sucesso em {output_file}')


Requisição bem-sucedida!
Dados salvos com sucesso em destiny_geodata.csv


In [178]:
destiny_geodata_df = pd.read_csv('destiny_geodata.csv', sep=',')
destiny_geodata_df.rename(columns={'delivery_city': 'destiny_city', 'delivery_suburb': 'destiny_suburb'}, inplace=True)
destiny_geodata_df.head()

,delivery_lng,delivery_lat,destiny_city,destiny_suburb
0,-48.116189,-15.848929,Ceilândia,P Sul
1,-48.118195,-15.850772,Ceilândia,P Sul
2,-48.112483,-15.847871,Ceilândia,P Sul
3,-48.118023,-15.846471,Ceilândia,P Sul
4,-48.114898,-15.858055,Sol Nascente/Pôr do Sol,Sol Nascente/Pôr do Sol


In [179]:
deliveries_df = pd.merge(left=deliveries_df, right=destiny_geodata_df[['destiny_suburb', 'destiny_city']], how='inner', left_index=True, right_index=True)
deliveries_df.head()

,name,region,origin_lat,origin_lng,origin_suburb,origin_city,vehicle_capacity,delivery_size,destiny_lat,destiny_lng,destiny_suburb,destiny_city
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,9,-15.848929,-48.116189,P Sul,Ceilândia
1,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,2,-15.850772,-48.118195,P Sul,Ceilândia
2,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,1,-15.847871,-48.112483,P Sul,Ceilândia
3,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,2,-15.846471,-48.118023,P Sul,Ceilândia
4,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,7,-15.858055,-48.114898,Sol Nascente/Pôr do Sol,Sol Nascente/Pôr do Sol


In [180]:
output_file = 'deliveries_loggi.csv'
deliveries_df.to_csv(output_file, sep=';', index=False)
print(f'Dados salvos com sucesso em {output_file}')

Dados salvos com sucesso em deliveries_loggi.csv


In [181]:
deliveries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636149 entries, 0 to 636148
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              636149 non-null  object 
 1   region            636149 non-null  object 
 2   origin_lat        636149 non-null  float64
 3   origin_lng        636149 non-null  float64
 4   origin_suburb     636149 non-null  object 
 5   origin_city       636149 non-null  object 
 6   vehicle_capacity  636149 non-null  int64  
 7   delivery_size     636149 non-null  int64  
 8   destiny_lat       636149 non-null  float64
 9   destiny_lng       636149 non-null  float64
 10  destiny_suburb    476264 non-null  object 
 11  destiny_city      634447 non-null  object 
dtypes: float64(4), int64(2), object(6)
memory usage: 58.2+ MB


In [182]:
deliveries_df.isna().any()

name                False
region              False
origin_lat          False
origin_lng          False
origin_suburb       False
origin_city         False
vehicle_capacity    False
delivery_size       False
destiny_lat         False
destiny_lng         False
destiny_suburb       True
destiny_city         True
dtype: bool

In [183]:
100 * (deliveries_df['destiny_city'].isna().sum() / len(deliveries_df))

np.float64(0.26754738276724477)

In [184]:
100 * (deliveries_df['destiny_suburb'].isna().sum() / len(deliveries_df))

np.float64(25.133262804783158)

In [185]:
prop_df = deliveries_df['destiny_city'].value_counts() / len(deliveries_df)
prop_df.sort_values(ascending=False).head(10)

destiny_city
Brasília                0.234195
Taguatinga              0.071625
Sudoeste / Octogonal    0.069504
Ceilândia               0.059661
Águas Claras            0.058293
Guará                   0.055619
Lago Sul                0.045620
Sobradinho              0.045412
Lago Norte              0.040832
Gama                    0.034880
Name: count, dtype: float64

In [186]:
prop_df = deliveries_df['destiny_suburb'].value_counts() / len(deliveries_df)
prop_df.sort_values(ascending=False).head(10)

destiny_suburb
Brasília            0.084564
Asa Norte           0.076639
Taguatinga          0.063363
Asa Sul             0.059263
Águas Claras        0.056598
Guará               0.055619
Samambaia           0.026514
Sobradinho          0.018148
Recanto das Emas    0.015029
P Sul               0.012807
Name: count, dtype: float64